<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [7]:
%pip install -Uqqq rich openai tiktoken wandb langchain unstructured tabulate pdf2image chromadb

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [9]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing. 

In [10]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens. 

In [19]:
MODEL_NAME = "text-davinci-003"
#MODEL_NAME = "gpt-4"

In [20]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('../docs_sample/')
len(documents)

11

In [21]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [22]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[2592, 310, 1154, 2135, 2330, 387, 763, 665, 1199, 2616, 2047]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [23]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(90, 438)

In [44]:
(document_sections[0].page_content) 

"import Tabs from '@theme/Tabs';\nimport TabItem from '@theme/TabItem';\n\nPyTorch Lightning\n\nPyTorch Lightning provides a lightweight wrapper for organizing your PyTorch code and easily adding advanced features such as distributed training and 16-bit precision. W&B provides a lightweight wrapper for logging your ML experiments. But you don't need to combine the two yourself: Weights & Biases is incorporated directly into the PyTorch Lightning library via the WandbLogger.\n\n⚡ Get going lightning-fast with just two lines.\n\n```python\nfrom pytorch_lightning.loggers import WandbLogger\nfrom pytorch_lightning import Trainer\n\nwandb_logger = WandbLogger()\ntrainer = Trainer(logger=wandb_logger)"

let's look at the first section

In [42]:
Markdown(document_sections[0].page_content)

import Tabs from '@theme/Tabs'; import TabItem from '@theme/TabItem';                                              

PyTorch Lightning                                                                                                  

PyTorch Lightning provides a lightweight wrapper for organizing your PyTorch code and easily adding advanced       
features such as distributed training and 16-bit precision. W&B provides a lightweight wrapper for logging your ML 
experiments. But you don't need to combine the two yourself: Weights & Biases is incorporated directly into the    
PyTorch Lightning library via the WandbLogger.                                                                     

⚡ Get going lightning-fast with just two lines.                                                                   

                                                                                                                   
 from pytorch_lightning.loggers import WandbLogger                                                                 
 from pytorch_lightning import Trainer                                                                             
                                                                                                                   
 wandb_logger = WandbLogger()                                                                                      
 trainer = Trainer(logger=wandb_logger)                                                                            
                                                                                                                   


## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query. 

In [45]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [46]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [53]:
query = "How can I share my W&B report with my team members in a public W&B project?"
docs = retriever.get_relevant_documents(query)

wandb: Streaming LangChain activity to W&B at https://wandb.ai/pepo/llmapps/runs/69ff01lk
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [56]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])

../docs_sample/collaborate-on-reports.md
../docs_sample/collaborate-on-reports.md
../docs_sample/teams.md


## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer. 

In [57]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [58]:
prompt

"Use the following pieces of context to answer the question at the end.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\ndescription: Collaborate and share W&B Reports with peers, co-workers, and your team.\n\nCollaborate on reports\n\nOnce you have saved a report, you can select the Share button to collaborate. A draft copy of the report is created when you select the Edit button. Draft reports auto-save. Select Save to report to publish your changes to the shared report.\n\nA warning notification will appear if an edit conflict occurs. This can occur if you and another collaborator edit the same report at the same time. The warning notification will guide you to resolve potential edit conflicts.\n\nComment on reports\n\nClick the comment button on a panel in a report to add a comment directly to that panel.\n\nWho can edit and share reports?\n\nReports that are created within an individual's private project is only visible to that user. 

In [59]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

To share a report in a public W&B project, select the Share button on the upper right hand corner. You can either  
provide an email account or copy the magic link. Users invited by email will need to log into Weights & Biases to  
view the report. Users who are given a magic link to not need to log into Weights & Biases to view the report.     
Shared reports are view-only.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [62]:
from langchain.chains import RetrievalQA

query = "How can I delete my project?"

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

Team admins can delete projects from the team settings page. They have full deletion permissions, including        
deleting runs, projects, artifacts, and sweeps.

In [63]:
import wandb
wandb.finish()